In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print(os.getcwd())

import time
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import Model
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import warnings
warnings.filterwarnings("ignore")

from dsvdd import *

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

from sklearn.metrics import roc_auc_score

/workspace/Linh/Autoencoder_SVDD
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def Vanilla_AE(H=5):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(9,)))
    model.add(keras.layers.Dense(6, activation='relu', use_bias=False))
    model.add(keras.layers.Dense(H, activation='relu', use_bias=False, name='code_layer'))
    model.add(keras.layers.Dense(6, activation='relu', use_bias=False))
    model.add(keras.layers.Dense(9, activation='tanh', use_bias=False))
    return model

In [3]:
def get_paysim():
    df = pd.read_csv('./data/PS_20174392719_1491204439457_log.csv')
    df = pd.concat([df,pd.get_dummies(df['type'], prefix='type_')], axis=1)
    df['hour'] = df['step'] % 24
    df['day'] = (df['step'] / 24 ).apply(np.ceil)
    df['weekday'] = df['day'] % 7
    df['weekday'] = df['weekday'].replace({0.0 : 7.0})
    df['sin_hour'] = np.sin(2*np.pi*df.hour/24)
    df['cos_hour'] = np.cos(2*np.pi*df.hour/24)
    df['sin_weekday'] = np.sin(2*np.pi*df.weekday/7)
    df['cos_weekday'] = np.cos(2*np.pi*df.weekday/7)
    df['type'] = LabelEncoder().fit_transform(df['type'])
    df['nameOrig'] = LabelEncoder().fit_transform(df['nameOrig'])
    df['nameDest'] = LabelEncoder().fit_transform(df['nameDest'])
    
    features = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'nameOrig', 'nameDest']
    features_values = df.loc[:, features].values
    scaler_values = StandardScaler().fit_transform(features_values)
    df[features] = PCA(n_components=len(features)).fit_transform(scaler_values)
    
    features_train = ['nameDest', 'oldbalanceOrg', 'newbalanceOrig', 'type', 'amount', 'sin_hour', 'day', 'step', 'hour']

    targets = ['isFraud']
    X = df.loc[:, features_train].values
    
    X = X - (X.max(0)+X.min(0))/2
    X = X/X.max(axis=0) 
    
    y = df.loc[:, targets].values
    y = y.reshape(-1,)
    y = np.where(y == 1, -1, 1)
    return X, y

In [4]:
X, y = get_paysim()

In [5]:
#ratio train-test 0.7 0.3
X_train, X_test , y_train, y_test = train_test_split(X, y, train_size=0.7)

In [5]:
#ratio train-val-test 0.6 0.2 0.2
X_train, X_test , y_train, y_test = train_test_split(X, y, train_size=8000, test_size=2000)
X_train, X_val , y_train, y_val = train_test_split(X_train, y_train, train_size=0.75)

In [33]:
tf.reset_default_graph()
AE_model = Vanilla_AE(H=5)
#pre_train_AE (not required)
AE_model.compile(optimizer='adam', loss='mse')
AE_model.fit(X_train, X_train, epochs=5, batch_size=128, validation_data=(X_test, X_test), verbose=True)

Train on 6000 samples, validate on 2000 samples
Epoch 1/5
6000/6000 [==============================] - 0s 29us/sample - loss: 0.4002 - val_loss: 0.3953
Epoch 2/5
6000/6000 [==============================] - 0s 12us/sample - loss: 0.3791 - val_loss: 0.3553
Epoch 3/5
6000/6000 [==============================] - 0s 12us/sample - loss: 0.2979 - val_loss: 0.2414
Epoch 4/5
6000/6000 [==============================] - 0s 12us/sample - loss: 0.1984 - val_loss: 0.1697
Epoch 5/5
6000/6000 [==============================] - 0s 12us/sample - loss: 0.1499 - val_loss: 0.1386


In [34]:
inputs = AE_model.input
outputs = AE_model.get_layer(name="code_layer").output
encoder = Model(inputs = inputs, outputs = outputs)

In [35]:
from dsvdd.utils import plot_most_normal_and_abnormal_images

# build model and DeepSVDD
svdd_soft = DeepSVDD(encoder, input_shape=(9,), representation_dim=5, objective='hard', nu=0.05)

t0 = time.time()
hisory_soft = svdd_soft.fit(X_train, X_train, y_train, epochs=10, verbose=True)
print('Train time:', time.time() - t0)

# test DeepSVDD
t0 = time.time()
score = svdd_soft.predict(X_test)
auc = roc_auc_score(y_test, -score)
print('Test time:', time.time() - t0)
print('------------- AUROC ----------- : %.4f' % auc)

  0%|          | 0/47 [00:00<?, ?it/s]

Epoch:   0 AUROC: 0.664
Epoch:   1 AUROC: 0.631
Epoch:   2 AUROC: 0.622
Epoch:   3 AUROC: 0.582


100%|██████████| 47/47 [00:00<00:00, 1621.61it/s]

Epoch:   4 AUROC: 0.606
Epoch:   5 AUROC: 0.620
Epoch:   6 AUROC: 0.641
Epoch:   7 AUROC: 0.634



100%|██████████| 47/47 [00:00<00:00, 1689.08it/s]


Epoch:   8 AUROC: 0.636
Epoch:   9 AUROC: 0.641
Train time: 0.676722526550293
Test time: 0.009450912475585938
------------- AUROC ----------- : 0.7637
